## Import Libraries

We imported all of the libraries needed for the project below!

In [1]:
# Import necessary libraries
import torch
import pyro
import numpy as np
import pandas as pd
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer import SVI, Trace_ELBO, Predictive
from pyro.optim import Adam
from pyro.infer.autoguide import AutoNormal

/opt/anaconda3/envs/cs179/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load and Prepare Data

We just read in the CSV file that was created earlier (``ca_restaurants_bayesian_dataset.csv``):
business_id,name,avg_rating,avg_sentiment_score,log_review_count

After loading the data into a Pandas DataFrame, we extract the columns named `avg_sentiment_score`, `log_review_count`, and `avg_rating` (these were really the only parts of the csv file we needed for our model). We then convert these columns into tensors so that they can be fed directly into our Pyro model. The tensor `X_sentiment` holds each restaurant’s sentiment score, `X_log_reviews` holds the log review counts, and `y` holds the target ratings we want to predict.


In [2]:
# Load the dataset
df = pd.read_csv("ca_restaurants_bayesian_dataset.csv")

# Extract the features and target
X_sentiment = torch.tensor(df["avg_sentiment_score"].values, dtype=torch.float)
X_log_reviews = torch.tensor(df["log_review_count"].values, dtype=torch.float)
y = torch.tensor(df["avg_rating"].values, dtype=torch.float)

## Define the Bayesian Model

The Bayesian linear regression model assumes that each restaurant’s rating just comes from a normal distribution whose mean depends on two inputs (the average review sentiment and the log review count).

We first assign prior distributions to each parameter:

- `alpha` is the intercept, drawn from a Normal(0, 1) prior.
- `beta_sentiment` is the slope for sentiment score, drawn from Normal(0, 1).
- `beta_reviews` is the slope for log review count, drawn from Normal(0, 1).
- `sigma` is the standard deviation of the noise, drawn from a HalfNormal(1) prior.

We then do `mean` = `alpha` + `beta_sentiment * X_sentiment` + `beta_reviews * X_log_reviews`. Each observed rating `y` is sampled from a Normal distribution centered at `mean` with standard deviation `sigma`. Pyro will just use this to compare model predictions against the actual ratings during inference.

In [3]:
# Define the Bayesian model
def bayesian_model(X_sentiment, X_log_reviews, y=None):
    # Define priors for the model parameters
    alpha = pyro.sample("alpha", dist.Normal(0.0, 1.0))
    beta_sentiment = pyro.sample("beta_sentiment", dist.Normal(0.0, 1.0))
    beta_log_reviews = pyro.sample("beta_reviews", dist.Normal(0.0, 1.0))
    sigma = pyro.sample("sigma", dist.HalfNormal(1.0))

    # Define the linear model
    mean = alpha + beta_sentiment * X_sentiment + beta_log_reviews * X_log_reviews
    
    # Sample from the likelihood
    with pyro.plate("data", len(X_sentiment)):
        pyro.sample("obs", dist.Normal(mean, sigma), obs=y)

## Create an Automatic Guide

This just automatically creates a normal distribution for each model parameter, allowing SVI to learn an approximate posterior without manually defining each variational distribution.

In [4]:
# Define the guide for mean-field variational inference
guide = AutoNormal(bayesian_model)

Fit the model with stochastic variational inference. Train the model with 5000 steps and print the loss.
## Train the Model with Variational Inference

Training process yay:

We set up an optimizer with a learning rate of 0.01 and tell Pyro to use the evidence lower bound as the loss function for variational inference. We then run a loop for 5000 steps. On each step Pyro updates the guide’s parameters to minimize the bound. Every 500 steps we also go ahead and print out the current loss so that we can track how training progresses toward convergence.


In [5]:
# Clear the parameter store
pyro.clear_param_store()

# Define the Stochastic Variational Inference (SVI) object
svi = SVI(
    model=bayesian_model,
    guide=guide,
    optim=Adam({"lr": 0.01}),
    loss=Trace_ELBO()
)

# Training the model with 5000 steps
num_steps = 5000
for step in range(num_steps):
    # Perform a single step of optimization
    loss = svi.step(X_sentiment, X_log_reviews, y)
    
    # Print the loss every 500 steps
    if step % 500 == 0:
        print(f"Step {step} : loss = {loss}")

Step 0 : loss = 5714.16952023888
Step 500 : loss = 529.0181514024734
Step 1000 : loss = 158.23740577697754
Step 1500 : loss = 82.58124113082886
Step 2000 : loss = 122.8308812379837
Step 2500 : loss = 80.42134487628937
Step 3000 : loss = 90.96460288763046
Step 3500 : loss = 83.69466906785965
Step 4000 : loss = 83.60319823026657
Step 4500 : loss = 84.94525158405304


## Inspect Posterior Samples

We examine the fitted model by drawing samples from the approx posterior distribution of each parameter.

Then once we have the samples, we just compute the mean and a 95 percent credible interval for each parameter by taking the 2.5th and 97.5th percentiles. Printing these values shows our best estimates for how sentiment and review count influence the predicted rating, along with how uncertain those estimates are.


In [6]:
# Extract the learned parameters
predictive = Predictive(
    bayesian_model,
    guide=guide,
    num_samples=1000,
    return_sites=["alpha", "beta_sentiment", "beta_reviews", "sigma"]
)

# Generate samples from the posterior predictive distribution
samples = predictive(X_sentiment, X_log_reviews)

for parameter in ["alpha", "beta_sentiment", "beta_reviews", "sigma"]:
    values = samples[parameter].detach().numpy()
    mean = values.mean()
    confidence_interval = (np.percentile(values, 2.5), np.percentile(values, 97.5))
    print(f"{parameter}: mean = {mean:.3f}, 95% confidence interval = {confidence_interval}")

alpha: mean = 1.951, 95% confidence interval = (np.float32(1.9303899), np.float32(1.9696169))
beta_sentiment: mean = 3.032, 95% confidence interval = (np.float32(3.0053725), np.float32(3.0571089))
beta_reviews: mean = -0.024, 95% confidence interval = (np.float32(-0.027778307), np.float32(-0.019841082))
sigma: mean = 0.264, 95% confidence interval = (np.float32(0.25002992), np.float32(0.27815536))


## Split Data into Training and Test Sets

We will randomly split it so that 80% of the examples are used for training and 20% for test. This ensures that our model is evaluated on data it has not seen during training.

In [7]:
# Determine the number of examples
n = X_sentiment.shape[0]

# Create a random permutation of indices from 0 to n-1
indices = torch.randperm(n)

# Split index at 80% of the data
train_size = int(0.8 * n)
train_idx = indices[:train_size]
test_idx = indices[train_size:]

# Create training tensors
X_train_sentiment = X_sentiment[train_idx]
X_train_log_reviews = X_log_reviews[train_idx]
y_train = y[train_idx]

# Create test tensors
X_test_sentiment = X_sentiment[test_idx]
X_test_log_reviews = X_log_reviews[test_idx]
y_test = y[test_idx]

# Print the sizes to confirm
print(f"Training set size: {train_size}")
print(f"Test set size: {n - train_size}")


Training set size: 547
Test set size: 137
